<a href="https://colab.research.google.com/github/saktheeswaranswan/yolo_excel_object_detection/blob/main/yolo_meme_engineering_and_yolo_speed_breaker_alarm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import pygame
import time

# Load YOLO model
net = cv2.dnn.readNet("yolov3-tiny.weights", "yolov3-tiny.cfg")
classes = ['person']
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

# Initialize camera and Pygame mixer
cap = cv2.VideoCapture(0)
pygame.mixer.init()

# Function to play MP3 file for 30 milliseconds
def play_mp3(file):
    pygame.mixer.music.load(file)
    pygame.mixer.music.play()
    time.sleep(5)
    pygame.mixer.music.stop()

while True:
    # Capture video frame
    ret, img = cap.read()
    height, width, channels = img.shape

    # Detect objects
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Display detections and play MP3 file for each class
    class_detected = {}
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                if classes[class_id] not in class_detected:
                    class_detected[classes[class_id]] = True
                    play_mp3(f"{class_id + 1}.mp3")

                color = colors[class_id]
                cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
                cv2.putText(img, classes[class_id], (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    cv2.imshow("Image", img)
    key = cv2.waitKey(1)
    if key == 27:
        break

cv2.destroyAllWindows()
pygame.mixer.quit()


In [ ]:
#for each class object detection corresponds each one mp3 file
#and select number of classes and put all mp3 file in one folde